In [96]:
# Cuts used to go from o.summary.fits to o.gst.fits

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np


In [97]:
path='test_photometry/'

In [98]:
cut_catalog=fits.open(path+'o.gst.fits')[1]
full_catalog=fits.open(path+'o.summary.fits')[1]

In [99]:
tbl_cut=Table(cut_catalog.data)
tbl_cut.colnames

['chip',
 'RA',
 'DEC',
 'X',
 'Y',
 'OBJECT_TYPE',
 'F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

In [100]:
tbl_full=Table(full_catalog.data)
tbl_full.colnames

['chip',
 'RA',
 'DEC',
 'X',
 'Y',
 'OBJECT_TYPE',
 'F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

In [101]:
sn2021sjt_position = SkyCoord(ra=309.33000691667*u.degree, dec=+66.106417016667*u.degree)

In [102]:
sn2021sjt_position

<SkyCoord (ICRS): (ra, dec) in deg
    (309.33000692, 66.10641702)>

In [103]:
catalog_radec = SkyCoord(ra=tbl['RA']*u.degree, dec=tbl['DEC']*u.degree)
test_object = SkyCoord(ra= 309.30106986096405*u.degree, dec=+66.12665483424358*u.degree)

In [104]:
# Now idx are indices into catalog that are the closest objects to each of the coordinates in c
# d2d are the on-sky distances between them,
# d3d are the 3-dimensional distances.
# Because coordinate objects support indexing, idx enables easy access to the matched set of coordinates 
# in the catalog:

#dra, ddec = sn2021sjt_position.spherical_offsets_to(matches)

In [105]:
search_radius = 1 * u.arcsec
#idx, d2d, d3d = sn2021sjt_position.match_to_catalog_sky(catalog)
d2d = sn2021sjt_position.separation(catalog_radec)
catalogmsk = d2d < search_radius
index = np.where(catalogmsk)[0]
print(index)
print(len(index))
objlist = np.zeros((len(index),4))
for i in range(len(index)):
    objlist[i][0]=tbl['RA'][index[i]]
    objlist[i][1]=tbl['DEC'][index[i]]
    objlist[i][2]=tbl['X'][index[i]]
    objlist[i][3]=tbl['Y'][index[i]]
print(objlist)

[3309 6656 6746 6753 7057]
5
[[ 309.32997059   66.10642008 2057.69       2455.34      ]
 [ 309.33034943   66.10635611 2072.79       2454.93      ]
 [ 309.33047245   66.10643374 2074.03       2446.64      ]
 [ 309.33067157   66.10635759 2083.55       2449.95      ]
 [ 309.32996709   66.10656332 2052.24       2443.52      ]]


In [106]:
tbl_cut['X']

334.81
3596.34
632.35
1053.19
1486.94
918.91
4028.09
755.28
128.97
...
2345.35


In [107]:
tbl_cut['Y'][3309]

2455.34

In [108]:
def writeregionfile(filename, objlist, color="blue",sys=''):
    if sys == '': sys = 'wcs'
    out = open(filename,'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+color+' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
      out.write('fk5\n')
      for ob in objlist:
        i += 1
        out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist[i][0], objlist[i][1], i))
    if sys == 'img':
      out.write('image\n')
      for ob in objlist:
        i += 1
        out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (objlist[i][2], objlist[i][3], i))
    out.close()

In [109]:
writeregionfile("near_stars_2021sjt_radec_" + str(search_radius), objlist, "red", "wcs")

In [110]:
writeregionfile("near_stars_2021sjt_pixels_" + str(search_radius), objlist, "red", "img")

In [120]:
#col = tbl_cut(, name=star)
#tab.add_column(col, index=3)
#tbl_cut.pprint_exclude_names.add('S')
tbl_cut[index].replace_column(tbl_cut[index]['F555W_VEGA'], 0)
print(tbl_cut[index]['F555W_VEGA', 'F555W_ERR', 'F555W_SNR', 'F555W_SHARP', 'F555W_ROUND', 'F555W_CROWD', 'F555W_FLAG'])

ValueError: column name F555W_VEGA
----------
    24.943
    26.939
    26.974
    26.988
    26.958 is not in the table

In [112]:
F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

SyntaxError: EOL while scanning string literal (1610768536.py, line 1)